In [1]:
#|default_exp process

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

# Install and load all libraries

In [7]:
#|export
from fastai.vision.all import *
from ml_utils import *
from skimage.measure import label,regionprops,find_contours
from evalutils import DetectionAlgorithm
from evalutils.validators import (UniquePathIndicesValidator, DataFrameValidator)
from evalutils.exceptions import ValidationError
from typing import (Tuple)
from typing import Dict
from pandas import DataFrame
import json, random, SimpleITK, gc, cv2, os, torch

In [8]:
from nbdev.export import nb_export

In [9]:
#|export
execute_in_docker = False

In [10]:
#|export
def fname_has_int(f):
    r = re.compile(r"\D*((?:\d+\.?)+)\D*")
    m = r.search(f.stem)
    if m is not None:
        print('There is int in the file name so no need to change file name.')
        return True
    else:
        print('Add int in the file name as currently there is no int in the file name.')
        return False

In [11]:
#|export
class VideoLoader():
    def load(self, *, fname):
        path = Path(fname).absolute()
        
        print(f'Loading file: {str(path)}')

        if not path.is_file():
            raise IOError(
                f"Could not load {fname} using {self.__class__.__qualname__}."
            )
        print (f'File {fname} found...')
        
        if not fname_has_int(path):
            old_file = path
            new_file = os.path.join(old_file.absolute().parent, str(old_file.stem)+'_1'+str(old_file.suffix))
            print(f'Old name: {old_file}')
            print(f'New name: {new_file}')
            os.rename(old_file, new_file)
            fname=new_file
            print(f'The file name is changed to contain an integer.')
        
        return [{"path": fname}]


    # only path valid
    def hash_video(self, input_video):
        pass

In [12]:
#|export
class UniqueVideoValidator(DataFrameValidator):
    """
    Validates that each video in the set is unique
    """

    def validate(self, *, df: DataFrame):
        try:
            hashes = df["video"]
        except KeyError:
            raise ValidationError("Column `video` not found in DataFrame.")

        if len(set(hashes)) != len(hashes):
            raise ValidationError(
                "The videos are not unique, please submit a unique video for "
                "each case."
            )

In [13]:
fs = get_files('./test/input/')
fs[0]
VideoLoader().load(fname=str(fs[0]))

Loading file: /home/bilal/mlworks/surgtoolloc2/test/input/vid_short_1.mp4
File test/input/vid_short_1.mp4 found...
There is int in the file name so no need to change file name.


[{'path': 'test/input/vid_short_1.mp4'}]

In [14]:
#|export
class Surgtoolloc_det(DetectionAlgorithm):
    def __init__(self):
        print(' ')
        print('TeamZERO prediction engine has started!.')
        print('Surgtoolloc_det.__init__() entered.')
        super().__init__(
            index_key='input_video',
            file_loaders={'input_video': VideoLoader()},
            input_path=Path("/input/") if execute_in_docker else Path("./test/input/"),
            output_file=Path("/output/surgical-tool-presence.json") if execute_in_docker else Path(
                "./test/output/surgical-tool-presence.json"),
            validators=dict(input_video=(UniquePathIndicesValidator(),)),
        )
        
        # loading ensemble learner
        print('-Loading models & tools dictionary.')
        self.cpu=False
        ensem_path=Path('/opt/algorithm/models') if execute_in_docker else Path("test/models")

        self.ensem_learner=[load_learner(m, cpu=self.cpu) for m in ensem_path.ls() if m.suffix=='.pkl']
        print(f'-{len(self.ensem_learner)} mutli-class classification models have been detected & loaded.')

        self.tool_list = ["needle_driver",
                          "monopolar_curved_scissor",
                          "force_bipolar",
                          "clip_applier",
                          "tip_up_fenestrated_grasper",
                          "cadiere_forceps",
                          "bipolar_forceps",
                          "vessel_sealer",
                          "suction_irrigator",
                          "bipolar_dissector",
                          "prograsp_forceps",
                          "stapler",
                          "permanent_cautery_hook_spatula",
                          "grasping_retractor"]
        print('-Tools dictionary loaded!.')
        print('Surgtoolloc_det.__init__() exited.')
        print(' ')
        
    def extract_images(self, video_file):     
        
        print('Surgtoolloc_det.extract_images(): entered.')
        # start the loop
        count = 0
        dst=Path('/images') if execute_in_docker else Path("./test/images/")
        for i in get_image_files(dst): os.remove(i) 
        
        # read the video file  
        print(f'-{str(video_file)} ready for frame extraction.')
        cap = cv2.VideoCapture(str(video_file))
        
        while True:
            is_read, f = cap.read()
            if not is_read:
                # break out of the loop if there are no frames to read
                break
            name = str(dst/f'im_{count}.jpg')
            cv2.imwrite(name,f)
            count+=1
        cap.release()
        print(f'-{len(get_image_files(dst))} images from {video_file} are extracted in {dst} folder. Extraction done!.')
        print('Surgtoolloc_det.extract_images(): exited.')
        
    def tool_detect_json_sample(self):
        # single output dict
        slice_dict = {"slice_nr": 1}
        tool_boolean_dict = {i: False for i in self.tool_list}
        single_output_dict = {**slice_dict, **tool_boolean_dict}
        return single_output_dict
        
    def process_case(self, *, idx, case):
        print('Surgtoolloc_det.process_case() entered')
        # Input video would return the collection of all frames (cap object)
        print(case)
        input_video_file_path = case #VideoLoader.load(case)
        scored_candidates = self.predict(case['path']) #video file > load evalutils.py

        # return
        # Write resulting candidates to result.json for this case
        print('Surgtoolloc_det.process_case() exited.')
        return scored_candidates

    def save(self):
        print('Surgtoolloc_det.save() entered')
        with open(str(self._output_file), "w") as f:
            json.dump(self._case_results[0], f)
        
        print('-file saved')
        print('Surgtoolloc_det.save() exited')
        
    def predict(self, fname) -> Dict:
        """
        Inputs:
        fname -> video file path
        
        Output:
        tools -> list of prediction dictionaries (per frame) in the correct format as described in documentation 
        """
        print('Surgtoolloc_det.predict() entered')
        print(f'Processing {str(fname)} for tools presence detection')
        print(' ')
        self.extract_images(fname)
        print(' ')

        images_dir = Path('/images') if execute_in_docker else Path("./test/images/")
        fs=get_image_files(images_dir)
        
        num_frames = len(fs)
        
        print(' ')
        print(f'-Tools presence detection task started.')

        # generate output json
        all_frames_predicted_outputs = []
        all_undefined_tools=[]
        
        tta_res=[]
        prs_items=[]
        for learn in self.ensem_learner:
            learn.dls.bs=16
            learn.dls.n_workers=2
            tta_res.append(learn.tta(dl=learn.dls.test_dl(fs)))
            if len(prs_items)<1:
                prs_items=learn.dl.items
            if not self.cpu:
                gc.collect()
                torch.cuda.empty_cache()

        print(f'-Predictions from all models in the ensemble learner are obtained!.')
        tta_prs=first(zip(*tta_res))
        tta_prs+=tta_prs[:1]
        tta_prs=torch.stack(tta_prs)

        lbls=[]
        for i in range(len(c)):
            arm_preds = tta_prs[:,:,cfg(i):cfg(i+1)].mean(0);
            arm_idxs = arm_preds.argmax(dim=1)
            arm_vocab = np.array(vocab[i])
            lbls.append(arm_vocab[arm_idxs])

        for usm1,usm2,usm3,usm4,f in zip(lbls[0],lbls[1],lbls[2],lbls[3],prs_items):
            frame_dict=self.tool_detect_json_sample()
            frame_dict['slice_nr']=int(f.stem.replace("im_",""))
            frame_dict[usm1]=True if usm1 in frame_dict.keys() else all_undefined_tools.append(usm1)
            frame_dict[usm2]=True if usm2 in frame_dict.keys() else all_undefined_tools.append(usm2)
            frame_dict[usm3]=True if usm3 in frame_dict.keys() else all_undefined_tools.append(usm3)
            frame_dict[usm4]=True if usm4 in frame_dict.keys() else all_undefined_tools.append(usm4)
            frame_dict.pop("nan", None)
            frame_dict.pop("blank", None)
            frame_dict.pop("out_of_view", None)
            all_frames_predicted_outputs.append(frame_dict) 
        
        print(f'-Translation of class probabilities to tool labels is done!.')
        
        print(f'-Following tools remained unaccounted for: {set(all_undefined_tools)}. Please ensure if it is OK to skip these tools from the output.')
        tools=sorted(all_frames_predicted_outputs, key=lambda d: d['slice_nr']) 

        print(f'-Output JSON file generated & returned!.')
        
        print(' ')
        
        print(f'{fname} has been successfully processed!.')
        print('Surgtoolloc_det.predict() exited')

        print(' ')
        return tools

In [15]:
%time pred_json=Surgtoolloc_det().predict(str(fs[0]))

 
TeamZERO prediction engine has started!.
Surgtoolloc_det.__init__() entered.
-Loading models & tools dictionary.
-3 mutli-class classification models have been detected & loaded.
-Tools dictionary loaded!.
Surgtoolloc_det.__init__() exited.
 
Surgtoolloc_det.predict() entered
Processing test/input/vid_short_1.mp4 for tools presence detection
 
Surgtoolloc_det.extract_images(): entered.
-test/input/vid_short_1.mp4 ready for frame extraction.
-60 images from test/input/vid_short_1.mp4 are extracted in test/images folder. Extraction done!.
Surgtoolloc_det.extract_images(): exited.
 
 
-Tools presence detection task started.


-Predictions from all models in the ensemble learner are obtained!.
-Translation of class probabilities to tool labels is done!.
-Following tools remained unaccounted for: {'nan'}. Please ensure if it is OK to skip these tools from the output.
-Output JSON file generated & returned!.
 
test/input/vid_short_1.mp4 has been successfully processed!.
Surgtoolloc_det.predict() exited
 
CPU times: user 1min 3s, sys: 43.3 s, total: 1min 46s
Wall time: 1min 11s


In [16]:
#|export
if __name__ == "__main__":
    Surgtoolloc_det().process()

 
TeamZERO prediction engine has started!.
Surgtoolloc_det.__init__() entered.
-Loading models & tools dictionary.
-3 mutli-class classification models have been detected & loaded.
-Tools dictionary loaded!.
Surgtoolloc_det.__init__() exited.
 
Key and File Loader
input_video <__main__.VideoLoader object at 0x7f24dbf52b50>
filter: None
Loading file: /home/bilal/mlworks/surgtoolloc2/test/input/vid_short_1.mp4
File test/input/vid_short_1.mp4 found...
There is int in the file name so no need to change file name.
test/input <__main__.VideoLoader object at 0x7f24dbf52b50> None
Inside the validate()
['input_video']
dict_keys(['input_video'])
input_video
input_video                          path
0  test/input/vid_short_1.mp4
Inside process_cases()...
Surgtoolloc_det.process_case() entered
path    test/input/vid_short_1.mp4
Name: 0, dtype: object
Surgtoolloc_det.predict() entered
Processing test/input/vid_short_1.mp4 for tools presence detection
 
Surgtoolloc_det.extract_images(): entered.
-te

-Predictions from all models in the ensemble learner are obtained!.
-Translation of class probabilities to tool labels is done!.
-Following tools remained unaccounted for: {'nan'}. Please ensure if it is OK to skip these tools from the output.
-Output JSON file generated & returned!.
 
test/input/vid_short_1.mp4 has been successfully processed!.
Surgtoolloc_det.predict() exited
 
Surgtoolloc_det.process_case() exited.
Surgtoolloc_det.save() entered
-file saved
Surgtoolloc_det.save() exited


In [17]:
nb_export('09_inference.ipynb', '.')

In [18]:
nb_export('09_inference.ipynb', '/home/bilal/mlworks/surgtoolloc2022-category-1')

In [19]:
!python -m process

 
TeamZERO prediction engine has started!.
Surgtoolloc_det.__init__() entered.
-Loading models & tools dictionary.
-3 mutli-class classification models have been detected & loaded.
-Tools dictionary loaded!.
Surgtoolloc_det.__init__() exited.
 
Key and File Loader
input_video <__main__.VideoLoader object at 0x7fc67fc94280>
filter: None
Loading file: /home/bilal/mlworks/surgtoolloc2/test/input/vid_short_1.mp4
File test/input/vid_short_1.mp4 found...
There is int in the file name so no need to change file name.
test/input <__main__.VideoLoader object at 0x7fc67fc94280> None
Inside the validate()
['input_video']
dict_keys(['input_video'])
input_video
input_video                          path
0  test/input/vid_short_1.mp4
Inside process_cases()...
Surgtoolloc_det.process_case() entered
path    test/input/vid_short_1.mp4
Name: 0, dtype: object
Surgtoolloc_det.predict() entered
Processing test/input/vid_short_1.mp4 for tools presence detection
 
Surgtoolloc_det.extract_images(): entered.
-te

In [ ]:
pred_json

In [ ]:
from fastai.test_utils import show_install
show_install()